# Restricted Facility Location

### `SMC` approach

In [1]:
# import libraries
from scipy.special import softmax

## basic imports 
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%config InlineBackend.figure_format = 'retina'
import warnings
warnings.filterwarnings('ignore')
from IPython.display import clear_output
import seaborn as sns
import cvxpy as cp
import time
clear_output()
myseed = '120824' 
np.random.seed(int(myseed))

In [2]:
## problem dimensions 

B = int(10) 
LAMBDA_extra = 0

TITLE = 'cr.csv'

### utils

In [3]:
def proj_simplex_vec(V, z=1):
    n_features = V.shape[1]
    U = np.sort(V, axis=1)[:, ::-1]
    z = np.ones(len(V)) * z
    cssv = np.cumsum(U, axis=1) - z[:, np.newaxis]
    ind = np.arange(n_features) + 1
    cond = U - cssv / ind > 0
    rho = np.count_nonzero(cond, axis=1)
    theta = cssv[np.arange(len(V)), rho - 1] / rho
    return np.maximum(V - theta[:, np.newaxis], 0)

# from https://github.com/cvxgrp/dccp/blob/master/dccp/linearize.py

def linearize(expr):
    """Returns the tangent approximation to the expression.

    Gives an elementwise lower (upper) bound for convex (concave)
    expressions. No guarantees for non-DCP expressions.

    Args:
        expr: An expression.

    Returns:
        An affine expression.
    """
    if expr.is_affine():
        return expr
    else:
        if expr.value is None:
            raise ValueError(
                "Cannot linearize non-affine expression with missing variable values."
            )
        tangent = np.real(expr.value) #+ np.imag(expr.value)
        grad_map = expr.grad
        for var in expr.variables():
            if grad_map[var] is None:
                return None
            complex_flag = False
            if var.is_complex() or np.any(np.iscomplex(grad_map[var])):
                complex_flag = True
            if var.ndim > 1:
                temp = cp.reshape(
                    cp.vec(var - var.value), (var.shape[0] * var.shape[1], 1)
                )
                if complex_flag:
                    flattened = np.transpose(np.real(grad_map[var])) @ cp.real(temp) + \
                    np.transpose(np.imag(grad_map[var])) @ cp.imag(temp)
                else:
                    flattened = np.transpose(np.real(grad_map[var])) @ temp
                tangent = tangent + cp.reshape(flattened, expr.shape)
            elif var.size > 1:
                if complex_flag:
                    tangent = tangent + np.transpose(np.real(grad_map[var])) @ (cp.real(var) - np.real(var.value)) \
                    + np.transpose(np.imag(grad_map[var])) @ (cp.imag(var) - np.imag(var.value))
                else:
                    tangent = tangent + np.transpose(np.real(grad_map[var])) @ (var - var.value)
            else:
                if complex_flag:
                    tangent = tangent + np.real(grad_map[var]) * (cp.real(var) - np.real(var.value)) \
                    + np.imag(grad_map[var]) * (cp.imag(var) - np.imag(var.value))
                else:
                    tangent = tangent + np.real(grad_map[var]) * (var - var.value) 
        return tangent

### true data generation

In [4]:
def preprocess(fname,col_locations,col_weight='population',col_city='city'):
    
    ## loading part 
    try:
        
        raw_df = pd.read_csv(fname)
        raw_df.dropna(inplace=True,axis=1)
        N = raw_df.shape[0]

        if isinstance(col_weight,str):
            weights = raw_df[col_weight].values
        elif isinstance(col_weight,numbers.Number):
            weights = raw_df.iloc[:,int(col_weight)].values
        else:
            print('col_weight should either be a column name (str) or location (int)')
            weights = np.ones(N)
            
        weights = np.maximum(weights,0.0)
        if sum(weights)>0:
            weights /= np.sum(weights)
            
        if isinstance(col_city,str):
            city_names = raw_df[col_city].values
        elif isinstance(col_city,numbers.Number):
            city_names = raw_df.iloc[:,int(col_city)].values
        else:
            print('col_city should either be a column name (str) or location (int)')
            city_names = ['city #'+str(elem+1) for elem in range(N)]
            
        assert isinstance(col_locations,list),'col_locations must be a list of names (str) or indices (int)'
        locations = np.zeros((N,len(col_locations)))
        for id_loc,col_loc in enumerate(col_locations):
            if isinstance(col_loc,str):
                buf = raw_df[col_loc].values
            elif isinstance(col_loc,numbers.Number):
                buf = raw_df.iloc[:,int(col_loc)].values
            else:
                print('erroneous prescribed marker for city location... 0 value set for the linked dim.')
                buf = np.zeros(N)
            locations[:,id_loc] = buf.copy()

    except:
        print('-> error while loading your file')
        return None,None,None
    
    return weights,locations,list(city_names)

In [5]:
weights,locations,city_names = preprocess(TITLE,col_locations=['lat','lng'],col_weight='population',col_city='city')

In [6]:
N_train = min(len(locations),int(750))
id_train = np.random.choice(np.arange(len(locations)),replace=False,size=N_train)

omegas_train = locations[id_train]
weights_train = weights[id_train]
w_ext = omegas_train.shape[1]

In [7]:
w_ext

2

In [8]:
N_train

183

### TRAINING

`parametric LP` 

In [9]:
# symmetry breakers tolerances
margin = .0
LAMBDA_extra = 10
mu_DCA = 1e-8

# variables
radius = cp.Variable(1)
centrum = cp.Variable(w_ext)
mat_cvx = cp.Variable((B,w_ext))

# parameters
param = cp.Parameter((N_train,B),nonneg=True)

# basic feasible set 
LBS,UBS = np.min(omegas_train,0),np.max(omegas_train,0)
LBS *= (1-np.sign(LBS)*.02)
UBS *= (1+np.sign(UBS)*.02)
r_ref = abs(1/4*np.mean(UBS-LBS))
X = [mat_cvx[e]<=UBS for e in range(B)]+[mat_cvx[e]>=LBS for e in range(B)] 

# prior-knowledge encoding
symmetry_breaks = []
if B>1:
    for e in range(B-1):
        symmetry_breaks += [cp.sum(mat_cvx[e])+margin<=cp.sum(mat_cvx[e+1])]
        
h_bar_smc = LAMBDA_extra*cp.maximum(0,radius[0]-r_ref) if LAMBDA_extra>0 else 0.0
F_param_smc = h_bar_smc 
f1_dc = h_bar_smc+mu_DCA/2*(cp.sum_squares(mat_cvx)+cp.sum_squares(centrum)+cp.sum_squares(radius))

# objective function implementation + centrum "close enough" constraint
for e in range(B):
    X += [cp.norm(mat_cvx[e]-centrum,'inf')<=radius[0]]
    f1_dc += weights_train@cp.multiply(np.ones(N_train),cp.sum(cp.abs(np.ones((N_train,1))@cp.reshape(mat_cvx[e],(1,w_ext))-omegas_train),1))
    F_param_smc += weights_train@cp.multiply(param[:,e].T,cp.sum(cp.abs(np.ones((N_train,1))@cp.reshape(mat_cvx[e],(1,w_ext))-omegas_train),1))

prob_param_smc = cp.Problem(cp.Minimize(F_param_smc),X+symmetry_breaks)

In [10]:
'''-> h components evaluation <-'''
def heval(mat_val,data=omegas_train,weights=weights_train):
    buf = np.zeros((len(data),B))
    for e in range(B):
        buf[:,e] = np.sum(np.abs(np.outer(np.ones(N_train),mat_val[e])-data),1)
    return np.outer(weights,np.ones(B))*buf

'''-> ERM evaluation<-'''
def Feval(radius,center,mat_val,data=omegas_train,weights=weights_train):
    values = heval(mat_val,data,weights)
    return LAMBDA_extra*max(0,radius[0]-r_ref) + np.sum(np.min(values,1))

In [11]:
'''-> computation of parameters <-
weights ndarray of shape (N_train,BB)
'''
def w2p(weights):
    param.value = weights
    
'''-> computation of f2_dc <-
weights ndarray of shape (N_train,BB) (opt)
'''
def w2l(weights):
    expr = 0.0
    loc_param = 1-weights
    for e in range(B):
        expr += weights_train@cp.multiply(loc_param[:,e].T,cp.sum(cp.abs(np.ones((N_train,1))@cp.reshape(mat_cvx[e],(1,w_ext))-omegas_train),1))
    return linearize(expr+mu_DCA/2*(cp.sum_squares(mat_cvx)+cp.sum_squares(centrum)+cp.sum_squares(radius)))
    
    
    
''' '''
def Q_opt(vals):
    buf = np.zeros(vals.shape)
    for s,sigma_s in enumerate(np.argmin(vals,1)):
        buf[s,sigma_s] += 1
    return buf

def Q_safe(vals,Q_current,Q_hat,Q_star,C):
    Q_next = np.zeros(Q_star.shape)
    combinations = np.zeros(len(Q_current))
    for _,Q_hat_elem in enumerate(Q_hat):
        num = np.sum((Q_current[_]-Q_star[_])*vals[_])
        denum = np.sum((Q_hat_elem-Q_star[_])*vals[_])
        if C*num>=denum:
            combinations[_] = 1
        else:
            combinations[_] = min(1,max(0,C*(num/max(1e-9,denum))))
        Q_next[_] = combinations[_]*Q_hat_elem+(1-combinations[_])*Q_star[_]
    return Q_next,combinations

In [12]:
NREPS = int(150)
Q_init_mats = []
for rep in range(NREPS):
    Q_init = np.random.uniform(0,1,(N_train,B))
    Q_init = np.array([q/np.sum(q) for q in Q_init])
    Q_init_mats.append(Q_init)

N_max = int(400)
tol = 1e-8

In [ ]:
'''AM'''

## init
f_am_overallbest = np.inf
fval_am = []
t_am = []
mat_am_out,center_am_out,radius_am_out = None,None,None
Q_am_out = None


print('=> am (START)')

for rep in range(NREPS):
    
    print('REPID #'+str(rep+1))
    print(' ')
    
    Q = Q_init_mats[rep].copy()
    w2p(Q)
    ref_val = np.inf
    
    t_init_am = time.time()
    
    try:

        ## main loop
        for k in range(N_max):

            #### set x 
            prob_param_smc.solve(solver=cp.MOSEK,warm_start=True)

            #### set weights
            components_ = heval(mat_cvx.value)
            prev = np.sum(components_/N_train*Q)
            Q = Q_opt(components_)
            w2p(Q)
            post = np.sum(components_/N_train*Q)
            F_Q_plus_x_plus = F_param_smc.value
            G_crit = max(0,prev-post)
            F_ = Feval(radius.value,centrum.value,mat_cvx.value)
            print("iter. %04d | Fval. %4.4e | BICval. %4.4e | Gcrit_x. %4.4e " % (k + 1, F_,F_Q_plus_x_plus,G_crit))
            if (ref_val-F_)<tol and k>1:
                print('=> convergence to criticality ... polishing phase')
                Q_final_am = Q_opt(heval(mat_cvx.value))
                w2p(Q_final_am)
                prob_param_smc.solve(solver=cp.MOSEK,warm_start=True)
                F_ = Feval(radius.value,centrum.value,mat_cvx.value)
                t_am.append(time.time()-t_init_am)
                fval_am.append(F_)
                print('final::: Fval. %4.4e | BICval. %4.4e' % (F_,prob_param_smc.value))
                print('=> am (END)')
                break;
            else:
                ref_val = F_Q_plus_x_plus
        if k==N_max-1:
            fval_am.append(Feval(radius.value,centrum.value,mat_cvx.value))
            t_am.append(time.time()-t_init_am)
    except: 
        print('... solver failed | early interruption ')
        fval_am.append(Feval(radius.value,centrum.value,mat_cvx.value))
        t_am.append(time.time()-t_init_am)
    print(' ')
    if fval_am[-1]<=f_am_overallbest:
        print(' /!\ NEW BEST | F = '+str(fval_am[-1])+'  /!\ ')
        f_am_overallbest = fval_am[-1]
        mat_am_out,center_am_out,radius_am_out = mat_cvx.value,centrum.value,radius.value
        Q_am_out = Q_final_am
        print(' ')

In [ ]:
'''Baratt-Boyd'''

def Q_hat_bb(vals,Q,param=1):
    minids = np.argmin(vals,1)
    Q_hat = Q.copy()
    for s,sigma_s in enumerate(minids):
        Q_hat[s] -= param*np.ones(vals.shape[1])
        Q_hat[s,sigma_s] += 2*param
    return proj_simplex_vec(Q_hat)

## init
f_bb_overallbest = np.inf
fval_bb = []
t_bb = []
mat_bb_out,center_bb_out,radius_bb_out = None,None,None
Q_bb_out = None


print('=> baratt-boyd (START)')

param_schedule = lambda k: 1e-1

for rep in range(NREPS):
    
    print('REPID #'+str(rep+1))
    print(' ')
    
    Q = Q_init_mats[rep].copy()
    w2p(Q)
    ref_val = np.inf
    
    t_init_bb = time.time()
    
    try:

        ## main loop
        for k in range(N_max):

            kappa = param_schedule(k)

            #### set x 
            prob_param_smc.solve(solver=cp.MOSEK,warm_start=True)

            #### set weights
            components_ = heval(mat_cvx.value)
            prev = np.sum(components_/N_train*Q)
            Q = Q_hat_bb(components_,Q,param=kappa)
            w2p(Q)
            post = np.sum(components_/N_train*Q)
            F_Q_plus_x_plus = F_param_smc.value
            G_crit = max(0,prev-post)
            F_ = Feval(radius.value,centrum.value,mat_cvx.value)
            print("iter. %04d | Fval. %4.4e | BICval. %4.4e | Gcrit_x. %4.4e " % (k + 1, F_,F_Q_plus_x_plus,G_crit))
            if (ref_val-F_)<tol and k>np.floor(1/param_schedule(0)): # /!\ otherwise early unwanted convergence /!\
                print('=> convergence to criticality ... polishing phase')
                Q_final_bb = Q_opt(heval(mat_cvx.value))
                w2p(Q_final_bb)
                prob_param_smc.solve(solver=cp.MOSEK,warm_start=True)
                F_ = Feval(radius.value,centrum.value,mat_cvx.value)
                t_bb.append(time.time()-t_init_bb)
                fval_bb.append(F_)
                print('final::: Fval. %4.4e | BICval. %4.4e' % (F_,prob_param_smc.value))
                print('=> maxmin (END)')
                break;
            else:
                ref_val = F_Q_plus_x_plus
        if k==N_max-1:
            fval_bb.append(Feval(radius.value,centrum.value,mat_cvx.value))
            t_bb.append(time.time()-t_init_bb)
    except: 
        print('... solver failed | early interruption ')
        fval_bb.append(Feval(radius.value,centrum.value,mat_cvx.value))
        t_bb.append(time.time()-t_init_bb)
    print(' ')
    if fval_bb[-1]<=f_bb_overallbest:
        print(' /!\ NEW BEST | F = '+str(fval_bb[-1])+'  /!\ ')
        f_bb_overallbest = fval_bb[-1]
        mat_bb_out,center_bb_out,radius_bb_out = mat_cvx.value,centrum.value,radius.value
        Q_bb_out = Q_final_bb
        print(' ')

In [ ]:
'''MAXMIN'''

def Q_hat_mm(vals,param=1):
    minvals,maxvals = np.min(vals,1),np.max(vals,1)
    return proj_simplex_vec(param*(np.outer(maxvals,np.ones(vals.shape[1]))-np.array(vals))/(np.outer(maxvals,np.ones(vals.shape[1]))-np.outer(minvals,np.ones(vals.shape[1]))))

## init
f_maxmin_overallbest = np.inf
fval_maxmin = []
t_maxmin = []
mat_maxmin_out,center_maxmin_out,radius_maxmin_out = None,None,None
Q_maxmin_out = None


print('=> maxmin (START)')

param_schedule = lambda k: k**(2/3) 
C_schedule = lambda k: 2/(k**(1/2)+3) # 0 for am

for rep in range(NREPS):
    
    print('REPID #'+str(rep+1))
    print(' ')
    
    Q = Q_init_mats[rep].copy()
    w2p(Q)
    ref_val = np.inf
    
    t_init_maxmin = time.time()
    
    try:

        ## main loop
        for k in range(N_max):

            C,kappa = C_schedule(k),param_schedule(k)

            #### set x 
            prob_param_smc.solve(solver=cp.MOSEK,warm_start=True)

            #### set weights
            components_ = heval(mat_cvx.value)
            prev = np.sum(components_/N_train*Q)
            Q_hat = Q_hat_mm(components_,param=kappa)
            Q_star = Q_opt(components_)
            Q,combis = Q_safe(components_,Q,Q_hat,Q_star,C)
            w2p(Q)
            post = np.sum(components_/N_train*Q)
            F_Q_plus_x_plus = F_param_smc.value
            G_crit = max(0,prev-post)
            F_ = Feval(radius.value,centrum.value,mat_cvx.value)
            print("iter. %04d | Fval. %4.4e | BICval. %4.4e | Gcrit_x. %4.4e | avg. epsilon. %4.4e" % (k + 1, F_,F_Q_plus_x_plus,G_crit,np.mean(combis)))
            if (ref_val-F_)<tol and k>1:
                print('=> convergence to criticality ... polishing phase')
                Q_final_maxmin = Q_opt(heval(mat_cvx.value))
                w2p(Q_final_maxmin)
                prob_param_smc.solve(solver=cp.MOSEK,warm_start=True)
                F_ = Feval(radius.value,centrum.value,mat_cvx.value)
                t_maxmin.append(time.time()-t_init_maxmin)
                fval_maxmin.append(F_)
                print('final::: Fval. %4.4e | BICval. %4.4e' % (F_,prob_param_smc.value))
                print('=> maxmin (END)')
                break;
            else:
                ref_val = F_Q_plus_x_plus
        if k==N_max-1:
            fval_maxmin.append(Feval(radius.value,centrum.value,mat_cvx.value))
            t_maxmin.append(time.time()-t_init_maxmin)
    except: 
        print('... solver failed | early interruption ')
        fval_maxmin.append(Feval(radius.value,centrum.value,mat_cvx.value))
        t_maxmin.append(time.time()-t_init_maxmin)
    print(' ')
    if fval_maxmin[-1]<=f_maxmin_overallbest:
        print(' /!\ NEW BEST | F = '+str(fval_maxmin[-1])+'  /!\ ')
        f_maxmin_overallbest = fval_maxmin[-1]
        mat_maxmin_out,center_maxmin_out,radius_maxmin_out = mat_cvx.value,centrum.value,radius.value
        Q_maxmin_out = Q_final_maxmin
        print(' ')

In [13]:
'''Softmin'''

def Q_hat_softmin(vals,param=1,num_pert_inf=5e-7):
    pert_vals = vals+np.random.uniform(-num_pert_inf,num_pert_inf,vals.shape)
    return softmax(-param*pert_vals/np.maximum(1e-4,np.abs(np.outer(np.mean(vals,1),np.ones(B)))),axis=1)

## init
f_sm_overallbest = np.inf
fval_sm = []
t_sm = []
mat_sm_out,center_sm_out,radius_sm_out = None,None,None
Q_sm_out = None


print('=> softmin (START)')

param_schedule = lambda k: (3/2)**(k**(3/4)) if (3/2)**(k**(3/4))<1e16 else np.inf
C_schedule = lambda k: 2/(k**(1/2)+3)

for rep in range(NREPS):
    
    print('REPID #'+str(rep+1))
    print(' ')
    
    Q = Q_init_mats[rep].copy()
    w2p(Q)
    ref_val = np.inf
    
    t_init_sm = time.time()
    
    try:

        ## main loop
        for k in range(N_max):
            
            C,kappa = C_schedule(k),param_schedule(k)
            

            #### set x 
            prob_param_smc.solve(solver=cp.MOSEK,warm_start=True)

            #### set weights
            components_ = heval(mat_cvx.value)
            prev = np.sum(components_/N_train*Q)
            Q_hat = Q_hat_softmin(components_,param=kappa)
            Q_hat = Q_hat/np.outer(np.sum(Q,1),np.ones(B))
            Q_star = Q_opt(components_)
            Q,combis = Q_safe(components_,Q,Q_hat,Q_star,C)
            w2p(Q)
            post = np.sum(components_/N_train*Q)
            F_Q_plus_x_plus = F_param_smc.value
            G_crit = max(0,prev-post)
            F_ = Feval(radius.value,centrum.value,mat_cvx.value)
            print("iter. %04d | Fval. %4.4e | BICval. %4.4e | Gcrit_x. %4.4e | avg. epsilon. %4.4e" % (k + 1, F_,F_Q_plus_x_plus,G_crit,np.mean(combis)))
            if (ref_val-F_)<tol and k>1:
                print('=> convergence to criticality ... polishing phase')
                Q_final_sm = Q_opt(heval(mat_cvx.value))
                w2p(Q_final_sm)
                prob_param_smc.solve(solver=cp.MOSEK,warm_start=True)
                F_ = Feval(radius.value,centrum.value,mat_cvx.value)
                t_sm.append(time.time()-t_init_sm)
                fval_sm.append(F_)
                print('final::: Fval. %4.4e | BICval. %4.4e' % (F_,prob_param_smc.value))
                print('=> softmin (END)')
                break;
            else:
                ref_val = F_Q_plus_x_plus
        if k==N_max-1:
            fval_sm.append(Feval(radius.value,centrum.value,mat_cvx.value))
            t_sm.append(time.time()-t_init_sm)
    except: 
        print('... solver failed | early interruption ')
        fval_sm.append(Feval(radius.value,centrum.value,mat_cvx.value))
        t_sm.append(time.time()-t_init_sm)
    print(' ')
    if fval_sm[-1]<=f_sm_overallbest:
        print(' /!\ NEW BEST | F = '+str(fval_sm[-1])+'  /!\ ')
        f_sm_overallbest = fval_sm[-1]
        mat_sm_out,center_sm_out,radius_sm_out = mat_cvx.value,centrum.value,radius.value
        Q_sm_out = Q_final_sm
        print(' ')

=> softmin (START)
REPID #1
 
iter. 0001 | Fval. 1.0199e+00 | BICval. 1.4988e+00 | Gcrit_x. 1.4440e-03 | avg. epsilon. 7.8259e-01
iter. 0002 | Fval. 9.7777e-01 | BICval. 1.2275e+00 | Gcrit_x. 1.3652e-03 | avg. epsilon. 4.4493e-01
iter. 0003 | Fval. 9.5542e-01 | BICval. 1.0775e+00 | Gcrit_x. 8.0731e-04 | avg. epsilon. 2.6695e-01
iter. 0004 | Fval. 8.4989e-01 | BICval. 9.2085e-01 | Gcrit_x. 7.0652e-04 | avg. epsilon. 3.3867e-01
iter. 0005 | Fval. 7.8313e-01 | BICval. 8.2005e-01 | Gcrit_x. 3.7066e-04 | avg. epsilon. 3.6152e-01
iter. 0006 | Fval. 7.8313e-01 | BICval. 7.9723e-01 | Gcrit_x. 1.2470e-04 | avg. epsilon. 2.9281e-01
iter. 0007 | Fval. 7.8261e-01 | BICval. 7.8790e-01 | Gcrit_x. 4.9802e-05 | avg. epsilon. 2.6358e-01
iter. 0008 | Fval. 7.8261e-01 | BICval. 7.8449e-01 | Gcrit_x. 1.8642e-05 | avg. epsilon. 2.4957e-01
iter. 0009 | Fval. 7.8261e-01 | BICval. 7.8326e-01 | Gcrit_x. 6.7159e-06 | avg. epsilon. 2.4508e-01
iter. 0010 | Fval. 7.8261e-01 | BICval. 7.8283e-01 | Gcrit_x. 2.3382e-

iter. 0026 | Fval. 5.6562e-01 | BICval. 5.6562e-01 | Gcrit_x. 1.4163e-09 | avg. epsilon. 4.9327e-02
iter. 0027 | Fval. 5.6562e-01 | BICval. 5.6562e-01 | Gcrit_x. 3.5494e-10 | avg. epsilon. 4.1936e-02
iter. 0028 | Fval. 5.6562e-01 | BICval. 5.6562e-01 | Gcrit_x. 8.7576e-11 | avg. epsilon. 3.9388e-02
iter. 0029 | Fval. 5.6562e-01 | BICval. 5.6562e-01 | Gcrit_x. 2.0408e-11 | avg. epsilon. 2.7684e-02
=> convergence to criticality ... polishing phase
final::: Fval. 5.6562e-01 | BICval. 5.6562e-01
=> softmin (END)
 
 /!\ NEW BEST | F = 0.565615876141491  /!\ 
 
REPID #4
 
iter. 0001 | Fval. 1.7462e+00 | BICval. 1.7931e+00 | Gcrit_x. 1.2985e-04 | avg. epsilon. 7.0994e-01
iter. 0002 | Fval. 8.7950e-01 | BICval. 1.2199e+00 | Gcrit_x. 1.9791e-03 | avg. epsilon. 6.3806e-01
iter. 0003 | Fval. 8.2328e-01 | BICval. 9.9041e-01 | Gcrit_x. 1.1024e-03 | avg. epsilon. 3.6510e-01
iter. 0004 | Fval. 6.6134e-01 | BICval. 7.8118e-01 | Gcrit_x. 9.0813e-04 | avg. epsilon. 3.4029e-01
iter. 0005 | Fval. 6.4079e-

iter. 0013 | Fval. 7.5059e-01 | BICval. 7.5121e-01 | Gcrit_x. 1.2877e-05 | avg. epsilon. 1.8009e-01
iter. 0014 | Fval. 7.4352e-01 | BICval. 7.4548e-01 | Gcrit_x. 2.5394e-05 | avg. epsilon. 1.9930e-01
iter. 0015 | Fval. 7.3675e-01 | BICval. 7.3833e-01 | Gcrit_x. 2.2987e-05 | avg. epsilon. 2.8647e-01
iter. 0016 | Fval. 6.6347e-01 | BICval. 6.6462e-01 | Gcrit_x. 3.2726e-04 | avg. epsilon. 4.6299e-01
iter. 0017 | Fval. 6.3756e-01 | BICval. 6.3777e-01 | Gcrit_x. 3.1792e-05 | avg. epsilon. 4.7779e-01
iter. 0018 | Fval. 6.3730e-01 | BICval. 6.3737e-01 | Gcrit_x. 1.0385e-06 | avg. epsilon. 4.5475e-01
iter. 0019 | Fval. 6.3708e-01 | BICval. 6.3715e-01 | Gcrit_x. 1.1882e-06 | avg. epsilon. 4.3780e-01
iter. 0020 | Fval. 6.3703e-01 | BICval. 6.3706e-01 | Gcrit_x. 3.3665e-07 | avg. epsilon. 4.2068e-01
iter. 0021 | Fval. 6.3703e-01 | BICval. 6.3704e-01 | Gcrit_x. 8.9330e-08 | avg. epsilon. 4.1001e-01
iter. 0022 | Fval. 6.3703e-01 | BICval. 6.3703e-01 | Gcrit_x. 2.6776e-08 | avg. epsilon. 3.2493e-01


iter. 0019 | Fval. 6.0150e-01 | BICval. 6.0150e-01 | Gcrit_x. 5.5029e-09 | avg. epsilon. 1.4612e-02
iter. 0020 | Fval. 6.0150e-01 | BICval. 6.0150e-01 | Gcrit_x. 1.5272e-09 | avg. epsilon. 1.6372e-02
iter. 0021 | Fval. 6.0150e-01 | BICval. 6.0150e-01 | Gcrit_x. 4.1985e-10 | avg. epsilon. 1.7159e-02
iter. 0022 | Fval. 6.0150e-01 | BICval. 6.0150e-01 | Gcrit_x. 1.1089e-10 | avg. epsilon. 6.0680e-03
iter. 0023 | Fval. 6.0134e-01 | BICval. 6.0134e-01 | Gcrit_x. 3.4390e-09 | avg. epsilon. 1.7304e-02
iter. 0024 | Fval. 6.0134e-01 | BICval. 6.0134e-01 | Gcrit_x. 5.3055e-10 | avg. epsilon. 1.7164e-02
iter. 0025 | Fval. 6.0134e-01 | BICval. 6.0134e-01 | Gcrit_x. 1.3297e-10 | avg. epsilon. 1.6821e-02
iter. 0026 | Fval. 6.0134e-01 | BICval. 6.0134e-01 | Gcrit_x. 2.6689e-11 | avg. epsilon. 1.6477e-02
=> convergence to criticality ... polishing phase
final::: Fval. 6.0134e-01 | BICval. 6.0134e-01
=> softmin (END)
 
REPID #10
 
iter. 0001 | Fval. 1.7624e+00 | BICval. 1.7995e+00 | Gcrit_x. 1.0330e-04

iter. 0010 | Fval. 6.0654e-01 | BICval. 6.0732e-01 | Gcrit_x. 8.5484e-06 | avg. epsilon. 3.1208e-02
iter. 0011 | Fval. 6.0529e-01 | BICval. 6.0576e-01 | Gcrit_x. 5.3737e-06 | avg. epsilon. 2.8555e-02
iter. 0012 | Fval. 6.0512e-01 | BICval. 6.0530e-01 | Gcrit_x. 2.1046e-06 | avg. epsilon. 1.3694e-02
iter. 0013 | Fval. 6.0394e-01 | BICval. 6.0404e-01 | Gcrit_x. 1.3346e-06 | avg. epsilon. 1.5178e-02
iter. 0014 | Fval. 6.0394e-01 | BICval. 6.0397e-01 | Gcrit_x. 4.1688e-07 | avg. epsilon. 1.2545e-02
iter. 0015 | Fval. 6.0394e-01 | BICval. 6.0395e-01 | Gcrit_x. 1.2733e-07 | avg. epsilon. 1.8173e-02
iter. 0016 | Fval. 6.0394e-01 | BICval. 6.0394e-01 | Gcrit_x. 3.8139e-08 | avg. epsilon. 3.4652e-02
iter. 0017 | Fval. 6.0394e-01 | BICval. 6.0394e-01 | Gcrit_x. 1.1151e-08 | avg. epsilon. 4.8148e-02
iter. 0018 | Fval. 6.0394e-01 | BICval. 6.0394e-01 | Gcrit_x. 3.1806e-09 | avg. epsilon. 4.7151e-02
iter. 0019 | Fval. 6.0394e-01 | BICval. 6.0394e-01 | Gcrit_x. 8.9459e-10 | avg. epsilon. 4.4789e-02


...

iter. 0026 | Fval. 6.0736e-01 | BICval. 6.0736e-01 | Gcrit_x. 7.2502e-09 | avg. epsilon. 6.2188e-02
iter. 0027 | Fval. 6.0736e-01 | BICval. 6.0736e-01 | Gcrit_x. 1.8174e-09 | avg. epsilon. 6.0843e-02
iter. 0028 | Fval. 6.0736e-01 | BICval. 6.0736e-01 | Gcrit_x. 4.4919e-10 | avg. epsilon. 4.6441e-02
iter. 0029 | Fval. 6.0736e-01 | BICval. 6.0736e-01 | Gcrit_x. 1.0936e-10 | avg. epsilon. 9.3489e-02
iter. 0030 | Fval. 6.0736e-01 | BICval. 6.0736e-01 | Gcrit_x. 2.6153e-11 | avg. epsilon. 4.3870e-02
=> convergence to criticality ... polishing phase
final::: Fval. 6.0736e-01 | BICval. 6.0736e-01
=> softmin (END)
 
REPID #137
 
iter. 0001 | Fval. 9.4426e-01 | BICval. 1.4546e+00 | Gcrit_x. 1.6268e-03 | avg. epsilon. 7.8935e-01
iter. 0002 | Fval. 8.0168e-01 | BICval. 1.1012e+00 | Gcrit_x. 1.6910e-03 | avg. epsilon. 5.5141e-01
iter. 0003 | Fval. 7.8313e-01 | BICval. 9.2312e-01 | Gcrit_x. 9.2974e-04 | avg. epsilon. 4.1030e-01
iter. 0004 | Fval. 7.8261e-01 | BICval. 8.4179e-01 | Gcrit_x. 4.4229e-0

iter. 0020 | Fval. 5.5812e-01 | BICval. 5.5812e-01 | Gcrit_x. 7.4187e-11 | avg. epsilon. 6.1167e-02
iter. 0021 | Fval. 5.5812e-01 | BICval. 5.5812e-01 | Gcrit_x. 1.9930e-11 | avg. epsilon. 6.0560e-02
=> convergence to criticality ... polishing phase
final::: Fval. 5.5812e-01 | BICval. 5.5812e-01
=> softmin (END)
 
REPID #140
 
iter. 0001 | Fval. 1.4425e+00 | BICval. 1.6273e+00 | Gcrit_x. 9.4750e-04 | avg. epsilon. 8.1178e-01
iter. 0002 | Fval. 8.3571e-01 | BICval. 1.1384e+00 | Gcrit_x. 1.7370e-03 | avg. epsilon. 6.3106e-01
iter. 0003 | Fval. 6.5013e-01 | BICval. 8.3574e-01 | Gcrit_x. 1.2583e-03 | avg. epsilon. 4.9042e-01
iter. 0004 | Fval. 6.3893e-01 | BICval. 7.1725e-01 | Gcrit_x. 6.0180e-04 | avg. epsilon. 3.8660e-01
iter. 0005 | Fval. 6.3845e-01 | BICval. 6.6894e-01 | Gcrit_x. 2.5919e-04 | avg. epsilon. 3.0179e-01
iter. 0006 | Fval. 6.3807e-01 | BICval. 6.4948e-01 | Gcrit_x. 1.0568e-04 | avg. epsilon. 2.4829e-01
iter. 0007 | Fval. 6.3791e-01 | BICval. 6.4199e-01 | Gcrit_x. 4.0417e-0

iter. 0027 | Fval. 6.2663e-01 | BICval. 6.2663e-01 | Gcrit_x. 3.1268e-08 | avg. epsilon. 8.2664e-02
iter. 0028 | Fval. 6.2663e-01 | BICval. 6.2663e-01 | Gcrit_x. 7.7518e-09 | avg. epsilon. 9.7309e-02
iter. 0029 | Fval. 6.2663e-01 | BICval. 6.2663e-01 | Gcrit_x. 1.8988e-09 | avg. epsilon. 6.6349e-02
iter. 0030 | Fval. 6.2663e-01 | BICval. 6.2663e-01 | Gcrit_x. 4.5875e-10 | avg. epsilon. 6.2868e-02
iter. 0031 | Fval. 6.2663e-01 | BICval. 6.2663e-01 | Gcrit_x. 1.0893e-10 | avg. epsilon. 5.0338e-02
iter. 0032 | Fval. 6.2585e-01 | BICval. 6.2585e-01 | Gcrit_x. 4.2749e-06 | avg. epsilon. 1.1737e-01
iter. 0033 | Fval. 6.2496e-01 | BICval. 6.2496e-01 | Gcrit_x. 1.4000e-06 | avg. epsilon. 1.0960e-01
iter. 0034 | Fval. 6.2494e-01 | BICval. 6.2494e-01 | Gcrit_x. 4.2208e-08 | avg. epsilon. 9.7279e-02
iter. 0035 | Fval. 6.2438e-01 | BICval. 6.2438e-01 | Gcrit_x. 2.2361e-06 | avg. epsilon. 1.1317e-01
iter. 0036 | Fval. 6.2383e-01 | BICval. 6.2383e-01 | Gcrit_x. 5.6907e-07 | avg. epsilon. 1.2570e-01


iter. 0009 | Fval. 6.1104e-01 | BICval. 6.1212e-01 | Gcrit_x. 1.1254e-05 | avg. epsilon. 2.7055e-02
iter. 0010 | Fval. 6.1001e-01 | BICval. 6.1061e-01 | Gcrit_x. 6.4756e-06 | avg. epsilon. 1.3913e-02
iter. 0011 | Fval. 6.0940e-01 | BICval. 6.0968e-01 | Gcrit_x. 3.2198e-06 | avg. epsilon. 8.7710e-03
iter. 0012 | Fval. 6.0940e-01 | BICval. 6.0949e-01 | Gcrit_x. 1.0573e-06 | avg. epsilon. 6.4362e-03
iter. 0013 | Fval. 6.0940e-01 | BICval. 6.0943e-01 | Gcrit_x. 3.3830e-07 | avg. epsilon. 6.4542e-03
iter. 0014 | Fval. 6.0940e-01 | BICval. 6.0941e-01 | Gcrit_x. 1.0567e-07 | avg. epsilon. 8.7839e-03
iter. 0015 | Fval. 6.0940e-01 | BICval. 6.0940e-01 | Gcrit_x. 3.2276e-08 | avg. epsilon. 1.5938e-02
iter. 0016 | Fval. 6.0940e-01 | BICval. 6.0940e-01 | Gcrit_x. 9.6934e-09 | avg. epsilon. 2.9585e-02
iter. 0017 | Fval. 6.0940e-01 | BICval. 6.0940e-01 | Gcrit_x. 2.8234e-09 | avg. epsilon. 4.0738e-02
iter. 0018 | Fval. 6.0940e-01 | BICval. 6.0940e-01 | Gcrit_x. 7.9415e-10 | avg. epsilon. 4.1609e-02


iter. 0016 | Fval. 6.2511e-01 | BICval. 6.2518e-01 | Gcrit_x. 1.1876e-06 | avg. epsilon. 1.1178e-01
iter. 0017 | Fval. 6.2506e-01 | BICval. 6.2509e-01 | Gcrit_x. 3.2017e-07 | avg. epsilon. 1.1391e-01
iter. 0018 | Fval. 6.2503e-01 | BICval. 6.2504e-01 | Gcrit_x. 1.3621e-07 | avg. epsilon. 9.2019e-02
iter. 0019 | Fval. 6.2503e-01 | BICval. 6.2503e-01 | Gcrit_x. 3.8490e-08 | avg. epsilon. 8.6753e-02
iter. 0020 | Fval. 6.2503e-01 | BICval. 6.2503e-01 | Gcrit_x. 1.0693e-08 | avg. epsilon. 8.4559e-02
iter. 0021 | Fval. 6.2503e-01 | BICval. 6.2503e-01 | Gcrit_x. 2.9224e-09 | avg. epsilon. 8.3455e-02
iter. 0022 | Fval. 6.2503e-01 | BICval. 6.2503e-01 | Gcrit_x. 7.8764e-10 | avg. epsilon. 8.2981e-02
iter. 0023 | Fval. 6.2503e-01 | BICval. 6.2503e-01 | Gcrit_x. 2.0880e-10 | avg. epsilon. 6.6373e-02
iter. 0024 | Fval. 6.2503e-01 | BICval. 6.2503e-01 | Gcrit_x. 5.6798e-11 | avg. epsilon. 6.2548e-03
iter. 0025 | Fval. 6.2503e-01 | BICval. 6.2503e-01 | Gcrit_x. 1.4915e-11 | avg. epsilon. 6.4014e-03
